## Evaluate the trained network (Week 12) - Step 6

####**Designed by Joon Son Chung, November 2020**

In this step, we convert the PyTorch model to TensorFlow Lite format so that they can be deployed on mobile devices and on the Coral board. We use post-training quantization in this exercise.





Change `pretrained_model` to the saved model that you want to convert.  We will use the validation set as the *representative dataset* for quantization.

In [ ]:
# mount Google Drive
from google.colab import drive, files
from zipfile import ZipFile
from PIL import Image
import os, glob, sys, shutil, time, numpy

drive.mount('/content/drive', force_remount=True)

# path of the data directory relative to the home folder of Google Drive
GDRIVE_HOME = '/content/drive/My Drive'

example_image     = os.path.join(GDRIVE_HOME,'MLVU/your_dataset/example.jpg')
pretrained_model  = os.path.join(GDRIVE_HOME,'MLVU/res18_vggface1_baseline.model')
val_zip           = os.path.join(GDRIVE_HOME,'MLVU/dataset4/val.zip') ## validation data as zip

with ZipFile(val_zip, 'r') as zipObj:
  zipObj.extractall("/val_set")

print('Validation files unzipped')

Install and import all necessary packages. The version of PyTorch and Tensorflow used here are `1.7.0` and `2.3.0` respectively. The code might not work with different versions of PyTorch and TF.

In [ ]:
! pip install --upgrade pip
! pip install onnx==1.8.0 
! pip install pytorch2keras==0.2.4 

import torch
import torchvision.transforms as transforms
import torchvision.models as models
from pytorch2keras.converter import pytorch_to_keras
import onnx
import tensorflow as tf

### Prepare PyTorch model and data
First, we define the model, which must be the same as the model trained.

In [ ]:
class FaceRecognition(torch.nn.Module):
	def __init__(self, nEmbed, nClasses):
	    super(FaceRecognition, self).__init__()
	    self.__S__ 	= models.resnet18(num_classes=nEmbed)
	    print('Initialised Softmax Loss')

	def forward(self, x, label=None):
		x 	= self.__S__(x)

		return x

Make example input.

In [ ]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Resize(256),
     transforms.CenterCrop([224,224]),
     transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])

img         = Image.open(example_image)
img_tensor  = transform(img).unsqueeze(0)

This script allows you to load parameters even if sizes of some weights have changed.

In [ ]:
def loadParameters(model, path):

    self_state = model.state_dict();
    loaded_state = torch.load(path, map_location="cuda:0");
    for name, param in loaded_state.items():
        origname = name;
        if name not in self_state:

            if name not in self_state:
                print("%s is not in the model."%origname);
                continue;

        if self_state[name].size() != loaded_state[origname].size():
            print("Wrong parameter length: %s, model: %s, loaded: %s"%(origname, self_state[name].size(), loaded_state[origname].size()));
            continue;

        self_state[name].copy_(param);


Load the PyTorch model.

In [ ]:
pt_model = FaceRecognition(nEmbed=512, nClasses=2700).cuda()
pt_model.eval()

loadParameters(pt_model,pretrained_model)

### PyTorch to Keras
Convert from PyTorch to Keras. `change_ordering` changes the data format from `NCHW` to `NHWC`, which is necessary for conversion to TF Lite.

In [ ]:
ONNX_PATH = "./my_model.onnx"
TF_PATH = "./my_tf_model.pb" # where the representation of tensorflow model will be stored
TFLITE_PATH = "./my_model.tflite"

k_model = pytorch_to_keras(pt_model, img_tensor.cuda(), [(3, 224, 224,)], change_ordering=True, verbose=False, name_policy='short')  

print('Converted to Keras')

Ensure that the output of the Keras model is the same as the PyTorch model.

In [ ]:
## Load image using PIL and Numpy
def numpy_loader(filename):
  image = Image.open(filename)
  image = image.resize((256,256),resample=Image.BILINEAR)
  image = image.crop((16,16,240,240))
  image = numpy.asarray(image, dtype=numpy.float32) / 255.
  image = numpy.subtract(image, numpy.array([0.485,0.456,0.406]))
  image = numpy.divide(image, numpy.array([0.229,0.224,0.225]))
  image = numpy.expand_dims(image, 0) 
  image = tf.cast(tf.convert_to_tensor(image), dtype=tf.float32)
  return image

## Inference Keras model
img_np  = numpy_loader(example_image)
x_tf    = k_model.predict(img_np)

## Inference PyTorch model
with torch.no_grad():
  x_pt = pt_model(img_tensor.cuda()).cpu().numpy()

## Check that the outputs are the same
from scipy import spatial
from numpy.linalg import norm

print('L2 norm (PT): %.4f'%numpy.linalg.norm(x_pt,2))
print('L2 norm (TF): %.4f'%numpy.linalg.norm(x_tf,2))
print('Cosine dist.: %.4f'%spatial.distance.cosine(x_pt, x_tf))

### Keras to TF Lite
Convert the Keras model to TF Lite. This can take a few minutes.

In [ ]:
def tf_loader(filename):
  image = tf.io.read_file(filename)
  image = tf.io.decode_jpeg(image, channels=3)
  image = tf.image.resize(image, [256, 256])
  image = tf.image.crop_to_bounding_box(image, 16, 16, 224, 224)
  image = tf.cast(image / 255., tf.float32)
  image = tf.subtract(image,[0.485,0.456,0.406])
  image = tf.divide(image,[0.229,0.224,0.225])
  image = tf.expand_dims(image, 0) 
  return image
  
TFLITE_PATH = "./my_model3.tflite"

# A generator that provides a representative dataset
def representative_data_gen():
  dataset_list = tf.data.Dataset.list_files('/val_set/*/*.jpg')
  for i in range(100):
    image = next(iter(dataset_list))
    image = tf_loader(image)
    yield [image]

# Convert the model
converter = tf.lite.TFLiteConverter.from_keras_model(k_model)
# converter = tf.compat.v1.lite.TFLiteConverter.from_keras_model_file(KERAS_PATH)
converter.experimental_new_converter = True
# This enables quantization
converter.optimizations = [tf.lite.Optimize.DEFAULT]
# This sets the representative dataset for quantization
converter.representative_dataset = representative_data_gen
# This ensures that if any ops can't be quantized, the converter throws an error
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
# For full integer quantization, though supported types defaults to int8 only, we explicitly declare it for clarity.
converter.target_spec.supported_types = [tf.int8]
# These set the input and output tensors to uint8 (added in r2.3)
converter.inference_input_type = tf.uint8
converter.inference_output_type = tf.uint8
tflite_model = converter.convert()

# Save the model.
with open(TFLITE_PATH, 'wb') as f:
  f.write(tflite_model)

print('Converted to TF Lite')

Try performing inference to ensure that the quantization has worked.

In [ ]:
def set_input_tensor(interpreter, input):
  input_details = interpreter.get_input_details()[0]
  tensor_index = input_details['index']
  input_tensor = interpreter.tensor(tensor_index)()[0]
  scale, zero_point = input_details['quantization']
  input_tensor[:, :] = numpy.uint8(input / scale + zero_point)
  
def infer(interpreter, input):
  set_input_tensor(interpreter, input)
  interpreter.invoke()
  output_details = interpreter.get_output_details()[0]
  output = interpreter.get_tensor(output_details['index'])
  # Outputs from the TFLite model are uint8, so we dequantize the results:
  scale, zero_point = output_details['quantization']
  output = numpy.array(output, dtype=numpy.float32)
  output = scale * (output - zero_point)
  return output

interpreter = tf.lite.Interpreter(model_content=tflite_model)
interpreter.allocate_tensors()

## test forward pass
x_tflite   = infer(interpreter, numpy_loader(os.path.join(data_dir,'example.jpg')))

## check cosine distance
print('L2 norm (TF): %.4f'%numpy.linalg.norm(x_tflite,2))
print('Cosine dist.: %.4f'%spatial.distance.cosine(x_tflite, x_tf))

## Compile for Edge TPU

First download the [Edge TPU Compiler](https://coral.ai/docs/edgetpu/compiler/)

In [ ]:
! curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | sudo apt-key add -

! echo "deb https://packages.cloud.google.com/apt coral-edgetpu-stable main" | sudo tee /etc/apt/sources.list.d/coral-edgetpu.list

! sudo apt-get update

! sudo apt-get install edgetpu-compiler	

Then compile the model for Edge TPU.

In [ ]:
! edgetpu_compiler my_model3.tflite

Download the converted model.

In [ ]:
files.download('my_model3_edgetpu.tflite')